In [1]:
from collections import defaultdict
import numpy as np

In [2]:
def read_input(fname):
    data = []
    max_z = 0
    with open(fname, 'r') as inf:
        for i, line in enumerate(inf.readlines()):
            if line.strip() == '':
                continue
            p_start, p_stop = line.strip().split('~')
            v = (i+1, tuple(int(v) for v in p_start.split(',')), tuple(int(v) for v in p_stop.split(',')))
            max_z = max(max_z, v[2][2])
            data.append(v)

    # data.sort(key=lambda z: z[2][2])
    # print('m', max_z)
    return sorted(data, key=lambda z: z[2][2]), max_z, i
    # return  data, max_z, i

In [3]:
def stack_cubes(data, max_z, nbricks):
    vol = np.zeros((10, 10, max_z), dtype=int)
    supports = defaultdict(set)
    supported = defaultdict(set)
    
    for i, pa, pe in data:
        # for z in range(0, 9):
        #     print(vol[0:3, 0:3, z])
        # print('Dropping', i, pa, pe)
        drop_vol = vol[pa[1]:pe[1]+1, pa[0]:pe[0]+1, :]
        # print('d', drop_vol)
        # print(np.where(drop_vol != 0))
        if np.any(drop_vol != 0):
            zmax = max(np.where(drop_vol != 0)[2])+1
        else:
            zmax = 0
                      
        # print('Dropping to', zmax)
        if zmax > 0:
            # for v in np.nditer(vol[pa[1]:pe[1]+1, pa[0]:pe[0]+1, zmax-1]):
            #     print(v)
            
            below = {int(v) for v in np.nditer(drop_vol[:, :, zmax-1])}
            # print('b', below, drop_vol[:, :, zmax-1])
            if 0 in below:
                below.remove(0)
            supported[i].update(below)
            for s in below:
                supports[s].add(i)

        # print('p', pa, pe)
        # print(pa[1],pe[1]+1, pa[0],pe[0]+1, zmax,zmax+(pe[2]-pa[2]) + 1)
        vol[pa[1]:pe[1]+1, pa[0]:pe[0]+1, zmax:zmax+(pe[2]-pa[2]) + 1] = i

    # print(supported)
    # print(supports)
    return supported, supports, nbricks

def can_be_removed(supported, supports, i):

    removable = True

    # print(i, supported[i])
    # if len(supported[i]) == 0 or len(supported[i]) > 1:
    #     removable = True

    # print('s', supports[i])
    for ni in supports[i]:
        # print('ni', ni, supported[ni])
        if len(supported[ni]) > 1:
            removable = True
        else:
            removable = False
            break
        # print('r', removable)

    # for ni in supports[i]:
    #     removable = can_be_removed(supported, supports, ni)

    # print(removable)
    return removable

def count_removable(supported, supports, nbricks):

    disintegratable = set()
    for i in range(1, nbricks+1):
        if can_be_removed(supported, supports, i):
            disintegratable.add(i)

    # print(disintegratable)
    return len(disintegratable)

def find_deps(supported, supports, i, deps):

    # print('f', i)
    for s in supported[i]:
        # print('c', s)
        if s not in deps:
            return deps

    # print('cont')
    deps.add(i)
    for s in supports[i]:
        deps = find_deps(supported, supports, s, deps)

    return deps
    
def chain_reaction(supported, supports, nbricks):

    n = 0
    for i in range(1, nbricks+1):
        deps = {i}
        for s in supports[i]:
            deps = find_deps(supported, supports, s, deps)
        deps.remove(i)
        # print(i, deps)
        n += len(deps)

    return n
    

In [4]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

n_removable = count_removable(*stack_cubes(*read_input('input22a.txt')))

print(f'# removable is {n_removable}')

assert n_removable == 5

print('\nPuzzle case\n')

n_removable = count_removable(*stack_cubes(*read_input('input22.txt')))

print(f'# removable is {n_removable}')

assert n_removable == 448

print('\n*****\nPuzzle2\n*****\n')

print('Test case\n')

n_falling = chain_reaction(*stack_cubes(*read_input('input22a.txt')))

print(f'Sum falling is {n_falling}')

assert n_falling == 7

print('\nPuzzle case\n')

n_falling = chain_reaction(*stack_cubes(*read_input('input22.txt')))

print(f'Sum falling is {n_falling}')

assert n_falling == 57770

*****
Puzzle1
*****

Test case

# removable is 5

Puzzle case

# removable is 448

*****
Puzzle2
*****

Test case

Sum falling is 7

Puzzle case

Sum falling is 57770
